In [ ]:
using LinearAlgebra
using PyPlot

In [ ]:
#Damped simple harmonic oscillator
k = 10.0; #spring constant
c = 0.2; #damping constant
h = 0.1; #10 Hz
A = exp(h*[0 1; -k -c])

In [ ]:
T = 50 #5 seconds

x0 = [1.0; 0]
xtraj = zeros(2,T);
xtraj[:,1] .= x0

for k = 1:(T-1)
    xtraj[:,k+1] .= A*xtraj[:,k];
end

In [ ]:
plot(xtraj[1,:])
plot(xtraj[2,:])

In [ ]:
#Generate a bunch of trajectories from random initial conditions
N = 10
H = zeros(2*T,N)
for ℓ = 1:N
    xtraj = zeros(2,T)
    xtraj[:,1] .= randn(2)
    for k = 1:(T-1)
        xtraj[:,k+1] .= A*xtraj[:,k]
    end
    H[:,ℓ] .= xtraj[:]
end

In [ ]:
#Pick some new initial condition
x0 = [-1; 1]

#Simulate with rollout
xtraj1 = zeros(2,T);
xtraj1[:,1] .= x0

for k = 1:(T-1)
    xtraj1[:,k+1] .= A*xtraj1[:,k];
end
xtraj1

In [ ]:
#Generate the same trajectory using the data matrix
E = [I zeros(2,2*(T-1))]
w = (H'*E'*E*H)\(H'*E'*x0)
z = H*w
xtraj2 = reshape(z,2,T)

In [ ]:
#The rollout matches the projection
plot(xtraj1[1,:])
plot(xtraj2[1,:])
xtraj1-xtraj2

In [ ]:
#Rank of the data matrix = dimension of the data manifold
rank(H)

In [ ]:
#Do the same thing with only positions y = [1 0]*x
C = [1.0 0]
N = 10
H = zeros(T,N)
for ℓ = 1:N
    xtraj = zeros(2,T)
    xtraj[:,1] .= randn(2)
    for k = 1:(T-1)
        xtraj[:,k+1] .= A*xtraj[:,k]
    end
    ytraj = C*xtraj;
    H[:,ℓ] .= ytraj[:];
end

In [ ]:
rank(H) #rank of data matrix is still the same

In [ ]:
#Pick some new initial condition
x0 = [1; -1]

#Simulate with rollout
xtraj1 = zeros(2,T);
xtraj1[:,1] .= x0

for k = 1:(T-1)
    xtraj1[:,k+1] .= A*xtraj1[:,k];
end
ytraj1 = (C*xtraj1)[:]

In [ ]:
#Take the first 2 positions and project onto data manifold
E = [I zeros(2,T-2)]
w = (H'*E'*E*H)\(H'*E'*ytraj1[1:2])
ytraj2 = H*w

In [ ]:
#Position trajectories match
plot(ytraj1)
plot(ytraj2)
ytraj1-ytraj2

In [ ]:
#Generate a random trajectory by "de-noising"
z̃ = randn(T)
plot(z̃)

In [ ]:
w = (H'*H)\(H'*z̃)
z = H*w
plot(z)

In [ ]:
#Now let's do a closed-loop sim with an LQR controller

using ControlSystems
Q = Array(1.0*I(2))
R = Array(0.1*I(1))

AB = exp(h*[0 1 0; -k -c 1; 0 0 0])
A = AB[1:2,1:2]
B = AB[1:2,3]

K = dlqr(A,B,Q,R)

In [ ]:
x0 = [1.0; 0]
xtraj = zeros(2,T);
utraj = zeros(T-1);
xtraj[:,1] .= x0
for k = 1:(T-1)
    utraj[k] = -(K*xtraj[:,k])[1]
    xtraj[:,k+1] .= (A-B*K)*xtraj[:,k];
end

plot(xtraj[1,:])
plot(xtraj[2,:])

In [ ]:
#Generate some random closed-loop trajectories
N = 10
H = zeros(3*T,N)
for ℓ = 1:N
    xtraj = zeros(2,T)
    xtraj[:,1] .= randn(2)
    utraj = zeros(T);
    utraj[:,1] .= -(K*xtraj[:,1])[1]
    for k = 1:(T-1)
        xtraj[:,k+1] .= A*xtraj[:,k] + B*utraj[k]
        utraj[k+1] = -(K*xtraj[:,k+1])[1]
    end
    H[:,ℓ] .= ([xtraj; utraj'])[:]
end

In [ ]:
rank(H) #rank of data matrix is still the same

In [ ]:
#Predict the closed-loop state-control trajectory from the first state using the data matrix
E = [I zeros(2,3*T-2)]
w = (H'*E'*E*H)\(H'*E'*x0)
ztraj = H*w

In [ ]:
#Pick off the first control
U = [0 0 1 zeros(1,3*T-3)]
u = U*ztraj

In [ ]:
#matches the LQR controller
-K*x0

In [ ]:
#We can even recover the LQR gain from the data matrix
-U*H*((H'*E'*E*H)\(H'*E'))

In [ ]:
K

In [ ]:
#What if we drive the system with white noise instead of using LQR?
N = 100
H = zeros(3*T-1,N)
for ℓ = 1:N
    xtraj = zeros(2,T)
    xtraj[:,1] .= randn(2)
    utraj = zeros(T-1);
    for k = 1:(T-1)
        utraj[k] = 0.1*randn();
        xtraj[:,k+1] .= A*xtraj[:,k] + B*utraj[k]
    end
    H[:,ℓ] .= [([xtraj[:,1:end-1]; utraj'])[:]; xtraj[:,end]]
end

In [ ]:
rank(H)

In [ ]:
#Do MPC with the data matrix
Nz = 3*T-1

Q = Diagonal([kron(ones(T-1),[1; 1; 1.0]); 1; 1])
α = 1e-4;
Y = [I zeros(2,147)]

#KKT matrix
F = factorize( [Q zeros(Nz,N) -I Y';
                zeros(N,Nz) α*I H' zeros(N,2);
                 -I H zeros(Nz,Nz) zeros(Nz,2);
                 Y zeros(2,N+Nz+2)]);

In [ ]:
#Closed-loop sim matches LQR
x0 = [1.0; 0]
xtraj = zeros(2,T);
utraj = zeros(T-1);
xtraj[:,1] .= x0
for k = 1:(T-1)
    #solve QP
    s = F\[zeros(149+100+149); xtraj[:,k]];

    utraj[k] = s[3]
    xtraj[:,k+1] .= A*xtraj[:,k] + B*utraj[k];
end

In [ ]:
plot(xtraj[1,:])
plot(xtraj[2,:])